In [8]:
import numpy as np
import pandas as pd

In [2]:
import joblib
import matplotlib.pyplot as plt

In [3]:
filename = "/Users/teliov/TUD/Thesis/Medvice/Notebooks/data/06_18_nlice_plus/nlice/data/output/rf/rf_serialized_sparse_nlice.joblib"

In [4]:
clf_data = joblib.load(filename)

In [5]:
clf = clf_data.get('clf')

In [6]:
train_file = "/Users/teliov/TUD/Thesis/Medvice/Notebooks/data/06_18_nlice_plus/nlice/data/parsed/train.csv_sparse.csv"

In [9]:
df = pd.read_csv(train_file, index_col='Index')

In [10]:
classes = df.LABEL.unique().tolist()

In [12]:
label_values = df.LABEL.values
ordered_keys = ['GENDER', 'RACE', 'AGE', 'SYMPTOMS']
df = df[ordered_keys]

In [13]:
from thesislib.utils.ml import models

In [14]:
import json

symptom_file = "/Users/teliov/TUD/Thesis/Medvice/Notebooks/data/06_18_nlice_plus/basic/symptom_db.json"
with open(symptom_file) as fp:
    symptom_db = json.load(fp)

In [15]:
num_symptoms = len(symptom_db)

In [16]:
sparsifier = models.ThesisAIMEDSymptomSparseMaker(num_symptoms=num_symptoms)

In [17]:
data_csc = sparsifier.fit_transform(df)

In [18]:
predicted_prob = clf.predict_proba(data_csc)

In [19]:
sort_idx = np.argsort(-predicted_prob, axis=1)

In [25]:
sorted_predicted_prob = -np.sort(-predicted_prob, axis=1)

In [27]:
predicted_labels = sort_idx[:, 0]

In [28]:
predicted_labels.shape

(153840,)

In [30]:
accuracy = np.sum(predicted_labels == label_values)/label_values.shape

In [32]:
correct_idx = np.where(predicted_labels == label_values)[0]

In [36]:
correct_idx.shape[0]/label_values.shape[0]

0.9647685907436298

In [39]:
wrong_idx = np.where(predicted_labels != label_values)[0]

In [40]:
wrong_idx.shape[0]/label_values.shape[0]

0.03523140925637026

In [48]:
# top 5
top_n_predictions = sort_idx[:, :5]
encoded_probability = np.take_along_axis(predicted_prob, label_values[:, None], axis=1)
encoded_probability = encoded_probability.reshape(encoded_probability.shape[0], )

In [49]:
bool_top_n = label_values.reshape(-1, 1) == top_n_predictions
bool_top_n = np.sum(bool_top_n, axis=1).astype(np.bool)
combined = np.logical_and(bool_top_n, (encoded_probability > 0))

In [50]:
np.sum(combined)/label_values.shape

array([0.99984399])